<a href="https://colab.research.google.com/github/stellarsb7/GFS-Net/blob/main/HistopathologicOralCancerIdentificationModelNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

# from google.colab import drive
# drive.mount('/content/drive')

! mkdir ~/.kaggle

# !cp /content/drive/MyDrive/ColabNotebooks/Kaggle_API/kaggle.json ~/.kaggle/kaggle.json
# !cp /content/drive/MyDrive/Colab_Notebooks/Kaggle_JSON/kaggle.json ~/.kaggle/kaggle.json



In [ ]:
# @title
# ! unzip /content/oral-cancer-dataset2-split-and-augmented.zip
# ! unzip /content/drive/MyDrive/KaggleData/histo_oral_cancer_full.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! unzip /content/drive/MyDrive/KaggleData/oralCancer2AugmentedSplit.zip

In [ ]:
! cp /content/kaggle.json  ~/.kaggle/kaggle.json

In [ ]:
! chmod 600   ~/.kaggle/kaggle.json

# ! kaggle datasets download -d sumanghosh1/oral-cancer-dataset2-split-and-augmented
! kaggle datasets download -d ashenafifasilkebede/histopathologic-oral-cancer-detection-using-cnns

In [ ]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import tensorflow as tf
from keras.metrics import Accuracy
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from tensorflow import keras
from tensorflow.keras.utils import plot_model
#from keras import backend as K

import os
import cv2
from PIL import Image
import numpy as np
from keras.utils import to_categorical

!pip install Py_FS
!pip install ReliefF
from  Py_FS.evaluation.evaluate import evaluate

In [ ]:
# @title Training Dataset
image_directory ="/content/Oral_Cancer_Split/Train_aug/"
SIZE = 224
X_train = []
y_train = []

X_val=[]
y_val=[]

cancerous = os.listdir(os.path.join(image_directory,'cancer'))
print(cancerous)
for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    if (image_name.split('.')[1] == 'jpg'):
        path=os.path.join(image_directory,'cancer',image_name)
        # print(path)
        image = cv2.imread(path)
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        if i<2130:
          X_train.append(np.array(image))
          y_train.append(1)
        else:
          X_val.append(np.array(image))
          y_val.append(1)

non_cancerous = os.listdir(os.path.join(image_directory,'non-cancer'))
for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    if (image_name.split('.')[1] == 'jpg'):
        image = cv2.imread(os.path.join(image_directory,'non-cancer',image_name))
        image = Image.fromarray(image, 'RGB')
        image = image.resize((SIZE, SIZE))
        if i<2130:
          X_train.append(np.array(image))
          y_train.append(0)
        else:
          X_val.append(np.array(image))
          y_val.append(0)


# image_directory = '/content/val/'
# SIZE = 224

# cancerous = os.listdir(image_directory + 'OSCC/')
# for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

#     if (image_name.split('.')[1] == 'jpg'):
#         image = cv2.imread(image_directory + 'OSCC/' + image_name)
#         image = Image.fromarray(image, 'RGB')
#         image = image.resize((SIZE, SIZE))
#         X_train.append(np.array(image))
#         y_train.append(1)

# non_cancerous = os.listdir(image_directory + 'Normal/')
# for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

#     if (image_name.split('.')[1] == 'jpg'):
#         image = cv2.imread(image_directory + 'Normal/' + image_name)
#         image = Image.fromarray(image, 'RGB')
#         image = image.resize((SIZE, SIZE))
#         X_train.append(np.array(image))
#         y_train.append(0)

print(len(X_train))
print(len(y_train))

print(len(X_val))
print(len(y_val))

In [ ]:
image_directory ="/content/Oral_Cancer_Split/Test/"
SIZE = 224
X_test = []
y_test = []

cancerous = os.listdir(os.path.join(image_directory,'cancer'))
print(cancerous)
for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    # if (image_name.split('.')[1] == 'jpg' or image_name.split('.')[1] == 'png' or image_name.split('.')[1] == 'jpeg'):
    path=os.path.join(image_directory,'cancer',image_name)
    # print(path)
    image = cv2.imread(path)
    image = Image.fromarray(image, 'RGB')
    image = image.resize((SIZE, SIZE))
    X_test.append(np.array(image))
    y_test.append(1)

non_cancerous = os.listdir(os.path.join(image_directory,'non-cancer'))
for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

    # if (image_name.split('.')[1] == 'jpg' or image_name.split('.')[1] == 'png' or image_name.split('.')[1] == 'jpeg'):
    image = cv2.imread(os.path.join(image_directory,'non-cancer',image_name))
    image = Image.fromarray(image, 'RGB')
    image = image.resize((SIZE, SIZE))
    X_test.append(np.array(image))
    y_test.append(0)

print(len(X_test))
print(len(y_test))

In [ ]:
# #@title Training Dataset
# image_directory ="/content/train/"
# SIZE = 224
# X_train = []
# y_train = []

# cancerous = os.listdir(os.path.join(image_directory,'OSCC'))
# for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

#     if (image_name.split('.')[1] == 'jpg'):
#         path=os.path.join(image_directory,'OSCC',image_name)
#         # print(path)
#         image = cv2.imread(path)
#         image = Image.fromarray(image, 'RGB')
#         image = image.resize((SIZE, SIZE))
#         X_train.append(np.array(image))
#         y_train.append(1)

# non_cancerous = os.listdir(os.path.join(image_directory,'Normal'))
# for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

#     if (image_name.split('.')[1] == 'jpg'):
#         image = cv2.imread(os.path.join(image_directory,'Normal',image_name))
#         image = Image.fromarray(image, 'RGB')
#         image = image.resize((SIZE, SIZE))
#         X_train.append(np.array(image))
#         y_train.append(0)


# image_directory = '/content/val/'
# SIZE = 224

# cancerous = os.listdir(os.path.join(image_directory,'OSCC'))
# for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

#     if (image_name.split('.')[1] == 'jpg'):
#         image = cv2.imread(image_directory + 'OSCC/' + image_name)
#         image = Image.fromarray(image, 'RGB')
#         image = image.resize((SIZE, SIZE))
#         X_train.append(np.array(image))
#         y_train.append(1)

# non_cancerous = os.listdir(os.path.join(image_directory,'Normal'))
# # non_cancerous = os.listdir(image_directory + 'Normal/')
# for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

#     if (image_name.split('.')[1] == 'jpg'):
#         image = cv2.imread(image_directory + 'Normal/' + image_name)
#         image = Image.fromarray(image, 'RGB')
#         image = image.resize((SIZE, SIZE))
#         X_train.append(np.array(image))
#         y_train.append(0)

# print(len(X_train))
# print(len(y_train))

In [ ]:
# #@title Test Dataset
# image_directory = '/content/test/'
# SIZE = 224
# X_test = []
# y_test = []

# cancerous = os.listdir(image_directory + 'OSCC/')
# for i, image_name in enumerate(cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

#     if (image_name.split('.')[1] == 'jpg'):
#         image = cv2.imread(image_directory + 'OSCC/' + image_name)
#         image = Image.fromarray(image, 'RGB')
#         image = image.resize((SIZE, SIZE))
#         X_test.append(np.array(image))
#         y_test.append(1)

# non_cancerous = os.listdir(image_directory + 'Normal/')
# for i, image_name in enumerate(non_cancerous):    #Remember enumerate method adds a counter and returns the enumerate object

#     if (image_name.split('.')[1] == 'jpg'):
#         image = cv2.imread(image_directory + 'Normal/' + image_name)
#         image = Image.fromarray(image, 'RGB')
#         image = image.resize((SIZE, SIZE))
#         X_test.append(np.array(image))
#         y_test.append(0)

# print(len(X_test))
# print(len(y_test))

In [ ]:
X_val=[]
y_val=[]

X_train2=[]
y_train2=[]

for i in range(len(X_train)):
  if i<240:
    X_val.append(X_train[i])
    y_val.append(y_train[i])
  else:
    X_train2.append(X_train[i])
    y_train2.append(X_train[i])

print(len(X_train2))
print(len(X_val))

In [ ]:
# final_model = tf.keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/Oral Cancer models/Copy of Copy of bm_nasnet_9286.h5')
final_model = tf.keras.models.load_model('/content/drive/MyDrive/Models/Oral-Cancer-Detection/Macro-Dataset2SplitAug/NoSqueeze.h5')
final_model.summary()

In [ ]:
INPUT_SHAPE = (224, 224, 3)
model = tf.keras.applications.NASNetMobile(input_shape = INPUT_SHAPE, include_top=True, weights='imagenet')
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Reshape, Multiply

def squeeze_excite_block(input_tensor, ratio=16):
    # Get the number of channels in the input tensor
    channels = input_tensor.shape[-1]

    # Squeeze: Global average pooling across the spatial dimensions
    squeeze = GlobalAveragePooling2D()(input_tensor)  # Reduce to 1x1 spatial dimensions
    squeeze = Reshape((1, 1, channels))(squeeze)     # Reshape to 1x1xC

    # Excitation: Two fully connected (dense) layers with ReLU activation
    excitation = Dense(channels // ratio, activation='relu')(squeeze)
    excitation = Dense(channels, activation='sigmoid')(excitation)

    # Reshape the excitation output to match the shape of the input tensor
    excitation = Reshape((1, 1, channels))(excitation)

    # Multiply the input tensor by the excitation output
    scaled_input = Multiply()([input_tensor, excitation])

    return scaled_input

In [ ]:
out=model.get_layer('activation_187').output
# out=squeeze_excite_block(out)

In [ ]:
out=model.get_layer('activation_187').output
out=squeeze_excite_block(out)

In [ ]:
output=GlobalAveragePooling2D()(out)
output=Dense(512,activation='relu')(output)
output=Dense(64,activation='relu')(output)
output=Dense(2,activation='sigmoid')(output)

In [ ]:
final_model=Model(inputs=model.input,outputs=output)

In [ ]:
final_model.summary()

In [ ]:
plot_model(final_model, show_shapes = True, show_layer_names = True)

In [ ]:
METRICS = ['accuracy',
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
           tf.keras.metrics.F1Score()
           ]
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/Models/Oral-Cancer-Detection/Macro-Dataset2SplitAug/Squeeze.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             verbose=1)
# final_model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',
#     metrics=METRICS
# )
lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(factor=0.001, patience=5, min_lr=1e-6)
history = final_model.fit(np.array(X_train),
              to_categorical(np.array(y_train)),
              batch_size = 8,
              validation_data = (np.array(X_val), to_categorical(np.array(y_val))),
              epochs = 20,
              callbacks=[checkpoint,lr_schedule]
              )

In [ ]:
final_model.evaluate(np.array(X_test), to_categorical(y_test))

In [ ]:
lab = np.array(y_train)
print(lab.shape)

In [ ]:
lab = lab.reshape(lab.shape[0], 1)
print(lab.shape)

In [ ]:
predictions = final_model.get_layer('global_average_pooling2d_1').output
model_feat = keras.Model(inputs = final_model.inputs ,outputs = predictions)
features = model_feat.predict(np.array(X_train))
print(features.shape)

In [ ]:
train_X = model_feat.predict(np.array(X_train))
train_Y = np.array(y_train)
test_X = model_feat.predict(np.array(X_test))
test_Y = np.array(y_test)

In [ ]:
from google.colab import runtime
runtime.unassign()